In [1]:
import pandas as pd
import numpy as np 


In [2]:

data = pd.read_csv("filtered_countries.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_edges = data[["countries_start", "countries_end"]]
data_edges = data_edges.rename(columns = {"countries_start" : "Source", "countries_end": "Target"})




In [4]:
data_edges

,Source,Target
0,United States,Aruba
1,Cayman Islands,Bermuda
2,China,Bermuda
3,Hong Kong,Bermuda
4,Hong Kong,Bermuda
5,China,Bermuda
6,Hong Kong,Bermuda
7,Singapore,Bermuda
8,United States,Bermuda
9,Hong Kong,Bermuda


In [5]:
data_counts = pd.DataFrame(data_edges.groupby(["Source","Target"]).size()).reset_index().rename(columns = {0: "count"})

In [6]:
data_counts["count"] = np.log(data_counts["count"])



In [7]:
data_edges = data_edges.drop_duplicates()

In [8]:
data_edges = data_edges.merge(data_counts, on = ["Source", "Target"], how = "left")

In [9]:
data_nodes = data[["countries_start"]]
data_nodes_2 = data[["countries_end"]]

data_nodes = data_nodes.rename(columns= {"countries_start": "Label"})

data_nodes_2 = data_nodes_2.rename(columns= {"countries_end": "Label"})



In [10]:
data_nodes = data_nodes.append(data_nodes_2)


In [11]:
data_nodes = data_nodes.drop_duplicates()

In [12]:
data_nodes = data_nodes.reset_index(drop = True)

In [13]:
geo = pd.read_csv("country_geo.csv")

In [14]:
data_nodes = data_nodes.merge(geo, left_on = "Label", right_on = "name", how = "left")


In [15]:
data_nodes[data_nodes["name"].isnull()]

,Label,country,latitude,longitude,name
102,Not identified,NaN,NaN,NaN,NaN


In [16]:
data_nodes = data_nodes[['Label', 'latitude', 'longitude']]
data_nodes = data_nodes.reset_index()
data_nodes = data_nodes.rename(columns = {'index': 'id'})


In [17]:
temp = data_edges.merge(data_nodes, left_on = 'Source', right_on = 'Label')
merged = temp.merge(data_nodes, left_on = 'Target', right_on = 'Label', suffixes = ('_source', '_target'))

In [18]:
merged = merged[['id_source', 'id_target', 'count']].rename(columns = {'id_source':'Source', 'id_target':'Target', 'count': "Weight"})


In [19]:

merged.to_csv("data_edges.csv", index= False)

In [20]:
data_nodes.to_csv("data_nodes.csv", index = False)